In [28]:
!pip install requests pandas numpy


In [30]:
import requests
import pandas as pd
import numpy as np


In [32]:
flight_number = []
mission_name = []
booster_version = []
payload_mass_list = []
orbit = []
launch_site = []
outcome = []
flights = []
gridfins = []
reused = []
legs = []
landing_pad = []
block = []
reused_count = []
longitude = []
latitude = []
serial = []


In [34]:
# Rocket
def getBoosterVersion(rocket_id):
    if rocket_id:
        response = requests.get(f"https://api.spacexdata.com/v4/rockets/{rocket_id}")
        if response.status_code == 200:
            booster_version.append(response.json().get('name'))
        else:
            booster_version.append(None)
    else:
        booster_version.append(None)

# Launchpad
def getLaunchSite(launchpad_id):
    if launchpad_id:
        response = requests.get(f"https://api.spacexdata.com/v4/launchpads/{launchpad_id}")
        if response.status_code == 200:
            data = response.json()
            launch_site.append(data.get('name'))
            longitude.append(data.get('longitude'))
            latitude.append(data.get('latitude'))
        else:
            launch_site.append(None)
            longitude.append(None)
            latitude.append(None)
    else:
        launch_site.append(None)
        longitude.append(None)
        latitude.append(None)

# Payload
def getPayloadData(payload_id_list):
    if payload_id_list:
        total_mass = 0
        orbit_name = None
        for payload_id in payload_id_list:
            response = requests.get(f"https://api.spacexdata.com/v4/payloads/{payload_id}")
            if response.status_code == 200:
                data = response.json()
                if data.get('mass_kg'):
                    total_mass += data['mass_kg']
                orbit_name = data.get('orbit')
        payload_mass_list.append(total_mass if total_mass > 0 else None)
        orbit.append(orbit_name)
    else:
        payload_mass_list.append(None)
        orbit.append(None)

# Core (Only first core)
def getCoreData(core_id_list):
    if core_id_list and isinstance(core_id_list, list) and len(core_id_list) > 0:
        core_id_entry = core_id_list[0]
        if core_id_entry and core_id_entry.get('core'):
            core_id = core_id_entry.get('core')
            response = requests.get(f"https://api.spacexdata.com/v4/cores/{core_id}")
            if response.status_code == 200:
                data = response.json()
                block.append(data.get('block'))
                reused.append(data.get('reused'))
                flights.append(data.get('flight'))
                gridfins.append(data.get('gridfins'))
                reused_count.append(data.get('reuse_count'))
                legs.append(data.get('legs'))
                landing_pad.append(data.get('landpad'))
                serial.append(data.get('serial'))
                outcome.append(data.get('landing_success'))
            else:
                block.append(None)
                reused.append(None)
                flights.append(None)
                gridfins.append(None)
                reused_count.append(None)
                legs.append(None)
                landing_pad.append(None)
                serial.append(None)
                outcome.append(None)
        else:
            block.append(None)
            reused.append(None)
            flights.append(None)
            gridfins.append(None)
            reused_count.append(None)
            legs.append(None)
            landing_pad.append(None)
            serial.append(None)
            outcome.append(None)
    else:
        block.append(None)
        reused.append(None)
        flights.append(None)
        gridfins.append(None)
        reused_count.append(None)
        legs.append(None)
        landing_pad.append(None)
        serial.append(None)
        outcome.append(None)


In [36]:
# SpaceX API URL
spacex_url = "https://api.spacexdata.com/v4/launches/past"

# Make GET request
response = requests.get(spacex_url)

# Convert response to JSON
launch_json = response.json()


In [38]:
for launch in launch_json:
    flight_number.append(launch.get('flight_number'))
    mission_name.append(launch.get('name'))
    getBoosterVersion(launch.get('rocket'))
    getLaunchSite(launch.get('launchpad'))
    getPayloadData(launch.get('payloads'))
    getCoreData(launch.get('cores'))


In [40]:
# Dictionary
launch_dict = {
    'Flight Number': flight_number,
    'Mission Name': mission_name,
    'Booster Version': booster_version,
    'Payload Mass (kg)': payload_mass_list,
    'Orbit': orbit,
    'Launch Site': launch_site,
    'Outcome': outcome,
    'Flights': flights,
    'Gridfins': gridfins,
    'Reused': reused,
    'Legs': legs,
    'Landing Pad': landing_pad,
    'Block': block,
    'Reused Count': reused_count,
    'Longitude': longitude,
    'Latitude': latitude,
    'Serial': serial
}

# DataFrame
launch_df = pd.DataFrame(launch_dict)

# Show first few rows
launch_df.head()


,Flight Number,Mission Name,Booster Version,Payload Mass (kg),Orbit,Launch Site,Outcome,Flights,Gridfins,Reused,Legs,Landing Pad,Block,Reused Count,Longitude,Latitude,Serial
0,1,FalconSat,Falcon 1,20.0,LEO,Kwajalein Atoll,None,None,None,None,None,None,NaN,0,167.743129,9.047721,Merlin1A
1,2,DemoSat,Falcon 1,NaN,LEO,Kwajalein Atoll,None,None,None,None,None,None,NaN,0,167.743129,9.047721,Merlin2A
2,3,Trailblazer,Falcon 1,NaN,LEO,Kwajalein Atoll,None,None,None,None,None,None,NaN,0,167.743129,9.047721,Merlin1C
3,4,RatSat,Falcon 1,165.0,LEO,Kwajalein Atoll,None,None,None,None,None,None,NaN,0,167.743129,9.047721,Merlin2C
4,5,RazakSat,Falcon 1,200.0,LEO,Kwajalein Atoll,None,None,None,None,None,None,NaN,0,167.743129,9.047721,Merlin3C


In [45]:
# Task 2: Filter for Falcon 9 launches
data_falcon9 = launch_df[launch_df['Booster Version'] != 'Falcon 1'].copy()

# Reset the Flight Number
data_falcon9.loc[:, 'Flight Number'] = list(range(1, data_falcon9.shape[0] + 1))

# Task 3: Dealing with missing values
# Calculate the mean value of 'Payload Mass (kg)'
payload_mass_mean = data_falcon9['Payload Mass (kg)'].mean()

# Replace NaN values in 'Payload Mass (kg)' with the mean
data_falcon9['Payload Mass (kg)'] = data_falcon9['Payload Mass (kg)'].replace(np.nan, payload_mass_mean)

# Export the cleaned data to CSV
data_falcon9.to_csv('dataset_part_1.csv', index=False)


In [47]:
data_falcon9.isnull().sum()


Flight Number          0
Mission Name           0
Booster Version        0
Payload Mass (kg)      0
Orbit                  1
Launch Site            0
Outcome              182
Flights              182
Gridfins             182
Reused               182
Legs                 182
Landing Pad          182
Block                  0
Reused Count           0
Longitude              0
Latitude               0
Serial                 0
dtype: int64